# Руководство по использованию класса SearchOscillograms

Этот Jupyter Notebook представляет собой руководство по использованию класса `SearchOscillograms` из модуля `osc_tools.data_management.search`. Класс `SearchOscillograms` предназначен для помощи в поиске, копировании и организации файлов осциллограмм.

Мы рассмотрим следующие ключевые методы:

1.  `copy_new_oscillograms`: Для копирования файлов осциллограмм из исходного каталога в целевой, с возможностью фильтрации по типу и предотвращения дубликатов с использованием хешей.
2.  `find_terminal_hashes_from_json`: Для идентификации хешей осциллограмм, связанных с определенными номерами терминалов, на основе JSON-файла, содержащего сопоставления хешей с путями.
3.  `organize_oscillograms_by_terminal`: Для упорядочивания файлов осциллограмм в структурированный формат каталогов, где файлы сортируются по подпапкам, названным в честь соответствующих терминалов.

**Примечание:** Перед запуском ячеек с кодом убедитесь, что класс `SearchOscillograms` доступен в вашей среде Python и что вы создали необходимые демонстрационные каталоги и файлы для корректной работы примеров.

## 1. `copy_new_oscillograms`

Этот метод используется для сканирования исходного каталога, идентификации различных типов файлов осциллограмм и их копирования в целевой каталог. Он может опционально сохранять исходную структуру каталогов и использует MD5-хеши для предотвращения копирования дубликатов файлов. Он также генерирует JSON-файлы (`_hash_table.json` и `new_hash_table_<timestamp>.json`) для отслеживания скопированных файлов.

### Параметры:

*   `source_dir` (str): Путь к каталогу, в котором находятся исходные файлы осциллограмм.
*   `dest_dir` (str): Путь к каталогу, куда будут скопированы выбранные файлы осциллограмм.
*   `types_to_copy` (list[TYPE_OSC], опционально): Список типов осциллограмм для копирования, используя перечисление `TYPE_OSC` (например, `[TYPE_OSC.COMTRADE_CFG_DAT, TYPE_OSC.BRESLER]`). Если параметр не указан (`None`), будут скопированы все поддерживаемые типы.
*   `copied_hashes` (dict, опционально): Словарь, содержащий хеши уже обработанных файлов. Это полезно для возобновления копирования или предотвращения повторной обработки. Для первого запуска это обычно пустой словарь `{}`. Если у вас есть `_hash_table.json` от предыдущего запуска, вы можете загрузить его сюда.
*   `preserve_dir_structure` (bool, опционально): Если `True` (по умолчанию), исходная структура папок из `source_dir` будет воспроизведена в `dest_dir`. Если `False`, файлы будут скопированы напрямую в `dest_dir` (или в подпапки для конкретных типов, например, `COMTRADE_CFG_DAT`).
*   `use_hashes` (bool, опционально): Если `True` (по умолчанию), MD5-хеши файлов `.dat` (для COMTRADE) или самих файлов (для других типов) используются для проверки дубликатов. Будут скопированы только новые файлы.

Другие параметры, такие как `_new_copied_hashes`, `_first_run`, `_path_temp`, `progress_callback`, `stop_processing_fn`, `is_write_names`, в основном предназначены для внутреннего использования или интеграции с графическими интерфейсами и обычно могут быть оставлены со значениями по умолчанию для базовых сценариев.

In [ ]:
import sys
import os

# Получаем текущий рабочий каталог ноутбука
notebook_dir = os.getcwd()

# Выход на исходный коталог
project_root = os.path.abspath(os.path.join(notebook_dir, os.pardir))

sys.path.append(project_root)
print(f"Добавлен путь в sys.path: {project_root}")

In [ ]:
import os
import shutil
import json
from osc_tools.data_management.search import SearchOscillograms
from osc_tools.core.constants import TYPE_OSC

# Исходный каталог с несколькими примерами файлов осциллограмм
source_example_dir = os.path.join(project_root, "raw_data","001_run_search_oscillograms_guide","")
# Целевой каталог для скопированных файлов
dest_example_dir = os.path.join(project_root, "raw_data","Output","")
# Путь для таблицы хешей (пример лежащий целевом каталоге)
hash_table_path = os.path.join(dest_example_dir, "_hash_table.json")

os.makedirs(source_example_dir, exist_ok=True)
os.makedirs(dest_example_dir, exist_ok=True)

# --- Создание экземпляра SearchOscillograms ---
search_ops = SearchOscillograms()

# --- Загрузка существующих хешей, если доступны (опционально) ---
# Для первого запуска copied_hashes будет {}
# При возобновлении вы можете загрузить ранее сохраненную таблицу хешей:
if os.path.exists(hash_table_path):
    with open(hash_table_path, 'r') as f:
        copied_hashes_example = json.load(f)
else:
    copied_hashes_example = {}

# --- Вызов copy_new_oscillograms ---
print(f"Запуск copy_new_oscillograms...")
print(f"Источник: {source_example_dir}")
print(f"Назначение: {dest_example_dir}")

# Пример: Копируем только файлы COMTRADE (.cfg/.dat) и CFF (.cff), сохраняя структуру
# Для этого передаем список нужных типов в параметр types_to_copy
types_to_select = [TYPE_OSC.COMTRADE_CFG_DAT, TYPE_OSC.COMTRADE_CFF]
# Если вы хотите скопировать все типы, можете передать types_to_copy=None или не указывать параметр
# types_to_select = None # Пример для копирования всех типов

print(f"Выбранные типы для копирования: {[t.name for t in types_to_select] if types_to_select else 'Все поддерживаемые типы'}")

count = search_ops.copy_new_oscillograms(
    source_dir=source_example_dir,
    dest_dir=dest_example_dir,
    copied_hashes=copied_hashes_example,
    preserve_dir_structure=True,
    use_hashes=True,
    types_to_copy=types_to_select
)

print(f"----------------------------------------------------")
print(f"copy_new_oscillograms завершен.")
print(f"Количество новых скопированных файлов: {count}")

## 2-3. `Вспомогательные функции`

Приведённые дальше функции являются вспомогательными для сортировки уже найденных папок по различным критериям.

## 2. `find_terminal_hashes_from_json`

Этот метод ищет хеши осциллограмм, связанные с определенными номерами терминалов. Он принимает JSON-файл (обычно `_hash_table.json`, созданный `copy_new_oscillograms`, или аналогичный файл, сопоставляющий хеши с путями и именами файлов) и список номеров терминалов. Затем он определяет, какие осциллограммы, вероятно, принадлежат этим терминалам, на основе:
1.  Имени файла: Если имя файла осциллограммы начинается с `t<номер_терминала_дополненный_до_5_цифр>` (например, `t00001.cfg`).
2.  Пути к файлу: Если путь к файлу содержит маркер "ОТГРУЖЕННЫЕ ТЕРМИНАЛЫ И ШКАФЫ" И номер терминала (дополненный до 5 или 4 цифр) в качестве имени каталога в пути.

Результатом является новый JSON-файл, сопоставляющий каждый указанный номер терминала (в виде строки) со списком связанных хешей осциллограмм.

### Параметры:

*   `input_json_path` (str): Путь к входному JSON-файлу. Этот файл должен иметь структуру типа: `{"хеш_значение": ["имя_файла.cfg", "полный/путь/к/имени_файла.cfg"], ...}`. Файл `_hash_table.json`, созданный `copy_new_oscillograms`, подходит для этого.
*   `terminal_numbers_to_find` (list[int]): Список целочисленных номеров терминалов, для которых вы хотите найти осциллограммы.
*   `output_json_path` (str): Путь, по которому будет сохранен результирующий JSON-файл. Формат вывода будет: `{"номер_терминала_как_строка": ["хеш1", "хеш2", ...], ...}`.

In [ ]:
import os
import json
from osc_tools.data_management.search import SearchOscillograms

# Целевой каталог для скопированных файлов
dest_example_dir = os.path.join(project_root, "raw_data.json", "Output", "")
# Путь для таблицы хешей (пример лежащий целевом каталоге)
hash_table_path = os.path.join(dest_example_dir, "_hash_table.json")


# --- Настройка для примера ---
# Предполагается, что 'output_data/copied_oscillograms/_hash_table.json' был создан на предыдущем шаге.
# Для этого примера мы создадим фиктивный _hash_table.json, если он не существует.

# Целевой каталог для скопированных файлов. Из предыдущего примера
dest_example_dir = os.path.join(project_root, "raw_data.json", "Output", "")
# Путь для таблицы хешей (пример лежащий целевом каталоге)
input_json_example_path = os.path.join(dest_example_dir, "_hash_table.json")
# Куда будут сохранены результаты
output_json_example_path = os.path.join(dest_example_dir, "terminal_hashes.json")

# Убедимся, что выходной каталог для terminal_hashes.json существует
os.makedirs(dest_example_dir, exist_ok=True)


print(f"Используемый входной JSON: {input_json_example_path}")
print(f"Выходной JSON будет: {output_json_example_path}")
print("----------------------------------------------------")

# --- Создание экземпляра SearchOscillograms ---
search_ops = SearchOscillograms() # Уже создан в предыдущей ячейке, если запускать последовательно

# --- Определение параметров ---
terminals_to_search_example = [1, 2, 3, 4] # Пример номеров терминалов

# --- Вызов find_terminal_hashes_from_json ---
print(f"Запуск find_terminal_hashes_from_json для терминалов: {terminals_to_search_example}...")
search_ops.find_terminal_hashes_from_json(
    input_json_path=input_json_example_path,
    terminal_numbers_to_find=terminals_to_search_example,
    output_json_path=output_json_example_path
)
print(f"----------------------------------------------------")
print(f"find_terminal_hashes_from_json завершен.")

if os.path.exists(output_json_example_path):
    print(f"Результаты сохранены в: {output_json_example_path}")
else:
    print(f"Ошибка: Выходной файл '{output_json_example_path}' не был создан.")

## 3. `organize_oscillograms_by_terminal`

Этот метод копирует файлы осциллограмм (в частности, `.cfg` и связанные с ними файлы `.dat`) из исходного каталога в структурированный целевой каталог. Целевой каталог будет содержать подпапки, названные в честь идентификаторов терминалов, и соответствующие файлы осциллограмм будут размещены в этих подпапках.

Этот метод опирается на список имён терминалов и словарь, который сопоставляет эти имена терминалов с идентификаторами осциллограмм (хешами или именами файлов), которые им принадлежат. Этот файл можно создать на предыдущем шаге.

### Параметры:

*   `source_dir` (str): Каталог, в котором в настоящее время хранятся файлы осциллограмм (например, `.cfg`, `.dat`). Часто это выходной каталог `copy_new_oscillograms`.
*   `dest_dir` (str): Корневой каталог, куда будут скопированы упорядоченные осциллограммы. Здесь будут созданы подкаталоги для каждого терминала.
*   `terminal_list` (list): Список строк, где каждая строка является идентификатором/именем терминала (например, `["Терминал_1", "Терминал_2"]`). Эти имена будут использоваться для создания подпапок в `dest_dir`.
*   `terminal_oscillogram_names` (dict): Словарь, который сопоставляет идентификаторы терминалов (из `terminal_list`) со списком идентификаторов осциллограмм.
    *   Пример: `{"Терминал_1": ["хешA", "хешB"], "Терминал_2": ["хешC"]}`
    *   Идентификаторы осциллограмм обычно представляют собой хеши (если `is_hashes=True`) или базовые имена файлов без расширений (если `is_hashes=False`). Этот словарь может быть получен из вывода `find_terminal_hashes_from_json`.
*   `is_hashes` (bool, опционально): По умолчанию `True`. Если `True`, идентификаторы осциллограмм в `terminal_oscillogram_names` рассматриваются как MD5-хеши (имя файла без расширения). Метод будет искать файлы `.cfg`, чье базовое имя совпадает с этими хешами в `source_dir`. Если `False`, идентификаторы рассматриваются как прямые базовые имена файлов (например, "osc1" для "osc1.cfg"), и метод будет вычислять хеш из соответствующего файла `.dat` для сопоставления с `terminal_oscillogram_names`. Описание проблемы подразумевает, что входные `terminal_oscillogram_names` обычно содержат хеши из `find_terminal_hashes_from_json`, поэтому `is_hashes=True` является типичным.

In [ ]:
import os
import shutil
import json
from osc_tools.data_management.search import SearchOscillograms

# --- Настройка для примера ---
# Этот пример предполагает:
# 1. Осциллограммы были скопированы с помощью `copy_new_oscillograms` в каталог,
#    например, project_root + "//raw_data//001_run_search_oscillograms_guide//" - Пример пути.
# 2. `find_terminal_hashes_from_json` создал JSON-файл, сопоставляющий номера терминалов с хешами,
#    например, 'raw_data//Output//terminal_hashes.json'.

# Исходный каталог, где находятся файлы .cfg/.dat (после copy_new_oscillograms)
# ВАЖНО: Измените этот путь, если ваш copy_new_oscillograms вывел файлы COMTRADE в определенную подпапку, например, "COMTRADE_CFG_DAT"
organize_source_dir = os.path.join(project_root, "raw_data", "001_run_search_oscillograms_guide","") # Пример пути

# Целевой каталог для упорядоченных файлов
organized_dest_dir = os.path.join(project_root, "raw_data", "Output","") 

# Путь к JSON-файлу из find_terminal_hashes_from_json
dest_example_dir = os.path.join(project_root, "raw_data.json", "Output", "") # Из предыдущего примера
terminal_hashes_json_path = os.path.join(dest_example_dir, "terminal_hashes.json")


# Убедимся, что каталоги существуют
os.makedirs(organize_source_dir, exist_ok=True)
os.makedirs(organized_dest_dir, exist_ok=True)

print(f"Источник для организации: {organize_source_dir}")
print(f"Целевой каталог для упорядоченных файлов: {organized_dest_dir}")
print(f"Используются хеши терминалов из: {terminal_hashes_json_path}")
print("----------------------------------------------------")

# --- Загрузка terminal_oscillogram_names из JSON-файла ---
try:
    with open(terminal_hashes_json_path, 'r', encoding='utf-8') as f:
        terminal_oscillogram_data = json.load(f)
except FileNotFoundError:
    print(f"Ошибка: JSON-файл с хешами терминалов не найден по пути '{terminal_hashes_json_path}'. Невозможно продолжить.")
    terminal_oscillogram_data = {} # Присваиваем пустой словарь, чтобы избежать ошибки позже, если пользователь захочет увидеть структуру

# Метод `organize_oscillograms_by_terminal` ожидает имена терминалов в качестве ключей.
# JSON из `find_terminal_hashes_from_json` использует номера терминалов (в виде строк) в качестве ключей.
# Мы можем использовать их напрямую, если считать номера терминалов их "именами" для папок.
# Или мы можем сопоставить их при необходимости, например, "Терминал_1", "Терминал_2". Для простоты используем номера как имена.
terminal_list_example = list(terminal_oscillogram_data.keys()) # ["1", "2", "3", "4"]
# terminal_oscillogram_names уже в правильном формате: {"строковый_id_терминала": [список_хешей]}

# !! Внимание !!
# Если что-то не получается - пустой результат, вероятно надо поменять is_hashes на True/False, 
# так как проблемы в именах файлов и то, заменены ли они или нет на хеш.
is_hashes = False

# --- Создание экземпляра SearchOscillograms ---
search_ops = SearchOscillograms() # Уже создан, если ячейки выполняются последовательно

# --- Вызов organize_oscillograms_by_terminal ---
print(f"Запуск organize_oscillograms_by_terminal...")
if terminal_oscillogram_data: # Продолжаем, только если данные были загружены
    search_ops.organize_oscillograms_by_terminal(
        source_dir=organize_source_dir, # Каталог, содержащий файлы <хеш>.cfg и <хеш>.dat
        dest_dir=organized_dest_dir,
        terminal_list=terminal_list_example,
        terminal_oscillogram_names=terminal_oscillogram_data, # Используем загруженные данные напрямую
        is_hashes=is_hashes 
    )
    print(f"----------------------------------------------------")
    print(f"organize_oscillograms_by_terminal завершен.")
else:
    print("Пропущен запуск organize_oscillograms_by_terminal из-за отсутствия данных о терминалах.")